In [1]:
import os
from googletrans import Translator

In [2]:
from bs4 import BeautifulSoup

In [3]:
os.chdir('html서식')

In [9]:
folder_list = os.listdir()

In [5]:
def cleansing(text):
    text = re.sub(re.compile('[(\s)]'), ' ', text)
    text = re.sub(re.compile('<'), '', text)
    text = re.sub(re.compile('>'), '', text)
    text = re.sub(re.compile('\s+'), ' ', text)
    return text if len(text) > 1 and re.match('[가-힣]', text) is not None else None


In [12]:
dir(os.path)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_get_sep',
 '_joinrealpath',
 '_varprog',
 '_varprogb',
 'abspath',
 'altsep',
 'basename',
 'commonpath',
 'commonprefix',
 'curdir',
 'defpath',
 'devnull',
 'dirname',
 'exists',
 'expanduser',
 'expandvars',
 'extsep',
 'genericpath',
 'getatime',
 'getctime',
 'getmtime',
 'getsize',
 'isabs',
 'isdir',
 'isfile',
 'islink',
 'ismount',
 'join',
 'lexists',
 'normcase',
 'normpath',
 'os',
 'pardir',
 'pathsep',
 'realpath',
 'relpath',
 'samefile',
 'sameopenfile',
 'samestat',
 'sep',
 'split',
 'splitdrive',
 'splitext',
 'stat',
 'supports_unicode_filenames',
 'sys']

In [13]:
file_dict = dict()
for f in folder_list:
    try:
        if os.path.isdir(f):
            with open('{}/index.xhtml'.format(f), 'r', encoding='utf-8') as f:
                content = BeautifulSoup(f.read(), 'lxml')
                lang_ko = content.select('span.lang-ko')
                file_dict[f] = lang_ko
    except FileNotFoundError or NotADirectoryError as e:
        continue

In [17]:
from sqlalchemy import Column, Integer, String, create_engine, ForeignKey, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base

In [18]:
from sqlalchemy.orm import sessionmaker
import re

In [19]:
engine = create_engine('sqlite:///corpus.db', echo=True)

In [20]:
base = declarative_base()

In [21]:
engine.table_names()

2020-08-26 13:33:57,361 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-26 13:33:57,362 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 13:33:57,364 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-26 13:33:57,365 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 13:33:57,366 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-08-26 13:33:57,367 INFO sqlalchemy.engine.base.Engine ()


['SEJONG_CORPUS']

In [24]:
class SejongCorpus(base):
    __tablename__ = 'SEJONG_CORPUS'
    
    pk = Column('pk', Integer, primary_key=True, autoincrement=True)
    ko_text = Column('ko_text', String, nullable=True)
    thai_text = Column('thai_text', String, nullable=True)
    file_name = Column('file_name', String, nullable=True)
    sent_count = Column('sent_count', Integer, nullable=True)
    cumsum_count = Column('cumsum_count', Integer, nullable=True)
    
    def __repr__(self):
        return 'pk:{}, ko_text:{}, thai_text:{}'.format(self.pk, self.ko_text, self.thai_text)

/Users/jaehyunlee/.virtualenvs/ko_university/lib/python3.7/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.SejongCorpus, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)


InvalidRequestError: Table 'SEJONG_CORPUS' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [25]:
Session = sessionmaker(bind=engine)
session = Session()

In [27]:
for k in file_dict.keys():
    for tag in file_dict[k]:
        cleansing_txt = cleansing(tag.text)
        if cleansing_txt is not None:
            ko_text = cleansing_txt
            translator = Translator()
            th_text = translator.translate(ko_text, dest='th').text
            obj = SejongCorpus(ko_text=ko_text, thai_text=th_text)
            session.add(obj)
            session.commit()

2020-08-26 13:36:25,116 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:36:25,118 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:36:25,119 INFO sqlalchemy.engine.base.Engine ('건축법 시행규칙 ', 'ข้อบังคับการบังคับใช้พระราชบัญญัติอาคาร', None, None, None)
2020-08-26 13:36:25,122 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:36:25,871 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:36:25,872 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:36:25,873 INFO sqlalchemy.engine.base.Engine ('별지 제14호서식', 'เอกสารแนบหมายเลข 14', None, None, None)
2020-08-26 13:36:25,883 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:36:26,457 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:36:26,458 INFO sqlalchemy.engine.base.Engi

2020-08-26 13:36:38,455 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:36:38,988 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:36:38,989 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:36:38,989 INFO sqlalchemy.engine.base.Engine ('연기사유', 'เหตุผลในการแสดง', None, None, None)
2020-08-26 13:36:38,994 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:36:39,537 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:36:39,538 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:36:39,539 INFO sqlalchemy.engine.base.Engine ('건축법', 'กฎหมายอาคาร', None, None, None)
2020-08-26 13:36:39,541 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:36:40,279 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:36:40,280 INFO sqla

2020-08-26 13:36:51,696 INFO sqlalchemy.engine.base.Engine ('근거법규', 'กฎหมายพื้นดิน', None, None, None)
2020-08-26 13:36:51,699 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:36:52,265 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:36:52,266 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:36:52,267 INFO sqlalchemy.engine.base.Engine ('건축법 ', 'กฎหมายอาคาร', None, None, None)
2020-08-26 13:36:52,269 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:36:52,979 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:36:52,980 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:36:52,981 INFO sqlalchemy.engine.base.Engine ('제14조', 'ข้อ 14', None, None, None)
2020-08-26 13:36:52,983 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:

2020-08-26 13:37:04,139 INFO sqlalchemy.engine.base.Engine ('영업장소', 'สถานที่ทำธุรกิจ', None, None, None)
2020-08-26 13:37:04,142 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:37:04,878 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:37:04,880 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:37:04,880 INFO sqlalchemy.engine.base.Engine ('업주명', 'ชื่อเจ้าของ', None, None, None)
2020-08-26 13:37:04,883 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:37:05,409 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:37:05,410 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:37:05,411 INFO sqlalchemy.engine.base.Engine ('건물구조', 'โครงสร้างอาคาร', None, None, None)
2020-08-26 13:37:05,416 INFO sqlalchemy.engine.base.Engine COMMIT
2020-

2020-08-26 13:37:17,995 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:37:17,996 INFO sqlalchemy.engine.base.Engine ('사건본인 ', 'กรณี', None, None, None)
2020-08-26 13:37:17,998 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:37:18,515 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:37:18,516 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 13:37:18,517 INFO sqlalchemy.engine.base.Engine ('청 구 취 지', 'วัตถุประสงค์ของการเรียกร้อง', None, None, None)
2020-08-26 13:37:18,520 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 13:37:19,066 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 13:37:19,067 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, 

...

2020-08-26 14:34:34,298 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:34:34,298 INFO sqlalchemy.engine.base.Engine ('의사 ', 'หมอ', None, None, None)
2020-08-26 14:34:34,302 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 14:34:34,928 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:34:34,930 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:34:34,931 INFO sqlalchemy.engine.base.Engine ('치과의사 ', 'ทันตแพทย์', None, None, None)
2020-08-26 14:34:34,937 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 14:34:35,837 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:34:35,837 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:34:35,

2020-08-26 14:34:49,302 INFO sqlalchemy.engine.base.Engine ('수수료 ', 'ค่าธรรมเนียม', None, None, None)
2020-08-26 14:34:49,304 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 14:34:50,004 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:34:50,005 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:34:50,005 INFO sqlalchemy.engine.base.Engine ('없음', 'ไม่มี', None, None, None)
2020-08-26 14:34:50,007 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 14:34:50,685 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:34:50,686 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:34:50,687 INFO sqlalchemy.engine.base.Engine ('행정정보 공동이용 동의서', 'ข้อตกลงในการใช้ข้อมูลการบริหารร่วมกัน', None, None, None)
2020-08-26 14:34:50,689 INFO sqlalchemy.engine.ba

2020-08-26 14:35:02,778 INFO sqlalchemy.engine.base.Engine ('사업자등록번호', 'หมายเลขทะเบียน บริษัท', None, None, None)
2020-08-26 14:35:02,779 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 14:35:03,316 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:35:03,317 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:35:03,317 INFO sqlalchemy.engine.base.Engine ('전화번호', 'หมายเลขโทรศัพท์', None, None, None)
2020-08-26 14:35:03,320 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 14:35:04,006 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:35:04,007 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:35:04,008 INFO sqlalchemy.engine.base.Engine ('주소', 'ที่อยู่', None, None, None)
2020-08-26 14:35:04,013 INFO sqlalchemy.engine.base.Engine COMMIT


2020-08-26 14:35:17,307 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:35:17,308 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:35:17,308 INFO sqlalchemy.engine.base.Engine ('폐관신고서', 'รายงานการปิด', None, None, None)
2020-08-26 14:35:17,311 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 14:35:17,831 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:35:17,832 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, thai_text, file_name, sent_count, cumsum_count) VALUES (?, ?, ?, ?, ?)
2020-08-26 14:35:17,833 INFO sqlalchemy.engine.base.Engine ('접수번호', 'ทะเบียนเลขที่', None, None, None)
2020-08-26 14:35:17,834 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-26 14:35:18,530 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 14:35:18,531 INFO sqlalchemy.engine.base.Engine INSERT INTO "SEJONG_CORPUS" (ko_text, t

ConnectTimeout: timed out

In [28]:
ko_corpus = [r[0] for r in session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all() if re.match(".*[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff].*", r[0]) is None]
thai_corpus = [r[1]+' ' for r in session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all() if re.match(".*[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff].*", r[0]) is None]

2020-08-26 18:56:53,195 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 18:56:53,211 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-26 18:56:53,219 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 18:56:59,888 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-26 18:56:59,888 INFO sqlalchemy.engine.base.Engine ()


In [29]:
len(ko_corpus), len(thai_corpus)

(482889, 482889)

In [31]:
with open('corpus.train.ko', 'w+', encoding='utf-8') as f:
    for ko in ko_corpus:
        f.write(ko+'|')
        
with open('corpus.train.th', 'w+', encoding='utf-8') as f:
    for th in thai_corpus:
        f.write(th+'|')